In [6]:
from __future__ import division
import os
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub import scipy_effects
from operator import itemgetter
from itertools import groupby, chain
# %matplotlib inline

In [2]:
def combine_to_range(power, power_threshold, sec_to_bin, min_silence_s):
    # power is the matrix produced by specgram
    # sec_to_bin is usally bin.shape[0] / audio length(ms)/1000
    
    idx_range = []
    for k, g in groupby(enumerate(np.where(power.sum(axis = 0) <= power_threshold)[0]), lambda (i,x):i-x):
        group = map(itemgetter(1), g)
        idx_range.append([group[0], group[-1]])
    idx_range = np.array(idx_range)
    silence_time_range = idx_range[np.where(((idx_range[:,1] 
                                              - idx_range[:,0])/sec_to_bin) > min_silence_s), :][0]/sec_to_bin
    silence_time_duration = silence_time_range[:,1] - silence_time_range[:,0]
    silence_time = sum(silence_time_duration)
    return silence_time_range, silence_time_duration, silence_time

In [3]:
def detect_silence(sound, power_threshold, min_silence_s):
    # power_threshold is a value that greater than 0. Usually is 1
    # min_silence_s is the minimum duration of a silence, in seconds. Usually is 3.
    
    # Get array of sound
    sample_rate = sound.frame_rate
    sound_track = sound.get_array_of_samples()
    sound_length = sound.duration_seconds
    
    # Plot specgram and get pxx, freq, bins, im
    pxx, _, bins, _ = plt.specgram(sound_track, scale = 'linear', Fs = sample_rate, NFFT = 512, noverlap = 256)
    plt.clf()
    plt.close('all')
#     plt.ylim(0,2000)
#     plt.xlabel('time(s)')
#     plt.ylabel('Frequency (Hz)')

    sec_to_bin = bins.shape[0] / sound_length
    
    
    si_time_duration = combine_to_range(pxx,power_threshold,sec_to_bin, min_silence_s)[1]
    active_rate = 1 - combine_to_range(pxx,power_threshold,sec_to_bin, min_silence_s)[2] / sound_length
    return si_time_duration, active_rate

In [33]:
path = os.getcwd() + "/AudioDownload/Tower/"
start_str = "KJFK-Twr-Apr-19-2017"
prefixed = [filename for filename in os.listdir(path) if filename.startswith(start_str)]
si_duration_agg = []
active_rates = []
for i in prefixed:
    print(i)
    sound = AudioSegment.from_mp3(path + i)
    si_duration, active_rate = detect_silence(sound, 1, 3)
    si_duration_agg.append(si_duration.tolist())
    active_rates.append(active_rate)

KJFK-Twr-Apr-19-2017-0000Z.mp3
KJFK-Twr-Apr-19-2017-0030Z.mp3
KJFK-Twr-Apr-19-2017-0100Z.mp3
KJFK-Twr-Apr-19-2017-0130Z.mp3
KJFK-Twr-Apr-19-2017-0200Z.mp3
KJFK-Twr-Apr-19-2017-0230Z.mp3
KJFK-Twr-Apr-19-2017-0300Z.mp3
KJFK-Twr-Apr-19-2017-0330Z.mp3
KJFK-Twr-Apr-19-2017-0400Z.mp3
KJFK-Twr-Apr-19-2017-0430Z.mp3
KJFK-Twr-Apr-19-2017-0500Z.mp3
KJFK-Twr-Apr-19-2017-0530Z.mp3
KJFK-Twr-Apr-19-2017-0600Z.mp3
KJFK-Twr-Apr-19-2017-0630Z.mp3
KJFK-Twr-Apr-19-2017-0700Z.mp3
KJFK-Twr-Apr-19-2017-0730Z.mp3
KJFK-Twr-Apr-19-2017-0800Z.mp3
KJFK-Twr-Apr-19-2017-0830Z.mp3
KJFK-Twr-Apr-19-2017-0900Z.mp3
KJFK-Twr-Apr-19-2017-0930Z.mp3
KJFK-Twr-Apr-19-2017-1000Z.mp3
KJFK-Twr-Apr-19-2017-1030Z.mp3
KJFK-Twr-Apr-19-2017-1100Z.mp3
KJFK-Twr-Apr-19-2017-1130Z.mp3
KJFK-Twr-Apr-19-2017-1200Z.mp3
KJFK-Twr-Apr-19-2017-1230Z.mp3
KJFK-Twr-Apr-19-2017-1300Z.mp3
KJFK-Twr-Apr-19-2017-1330Z.mp3
KJFK-Twr-Apr-19-2017-1400Z.mp3
KJFK-Twr-Apr-19-2017-1430Z.mp3
KJFK-Twr-Apr-19-2017-1500Z.mp3
KJFK-Twr-Apr-19-2017-1530Z.mp3
KJFK-Twr

In [47]:
daily_gap = list(chain.from_iterable(si_duration_agg))
plt.hist(daily_gap, bins = range(0, min(int(max(daily_gap))+20, 180), 10))
plt.title('Histogram of silence duration for ' + start_str[9:])
plt.xlabel('Silence Duration(second)')
plt.ylabel('Count')
plt.ylim(0,1200)
plt.show()

In [46]:
plt.plot(np.linspace(0,24,len(active_rates)), active_rates)
plt.title('Active Rate of ' + start_str[9:])
plt.xticks(range(25), range(0,25))
plt.xlabel('UTC Hour')
plt.ylabel('Active Rate')
# plt.ylim(0,25)
plt.show()